In [215]:
!pip install webdriver_manager

In [216]:
# imports
import requests as rq
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from selenium import webdriver
import os
from webdriver_manager.chrome import ChromeDriverManager
import unicodedata
# end imports

# Part 01  : Data Gathering

In [217]:
# Settings 
chrome_driver_path = '/Users/chihab_eddine98/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver'

reviews_url = 'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/product-reviews/B07X8NL7MW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

reviews_div_cls = 'review'

prod_url = 'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/dp/B07X8NL7MW/ref=sr_1_7?__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=Microsoft+Surface&qid=1616930888&s=computers&sr=1-7'

prod_urls = {'informatique':[
    {
        'prod_url':
        'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/dp/B07X8NL7MW/ref=sr_1_7? __mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=Microsoft+Surface&qid=1616930888&  s=computers&sr=1-7',
    'rev_url': 'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/product-reviews/B07X8NL7MW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
    
    }
    ]
}



# End Settings

# Scrapping Product Information

In [218]:

# Scrapping Product Data 
def product_scrapper(driver,prod_category,prod_url):
    driver.get(prod_url)
    prod_soup = BeautifulSoup(driver.page_source,'html.parser')

    # Product Info
    prod_title = prod_soup.find('span',{'id' : 'productTitle' }).text
    prod_brand = prod_soup.find('a',{'id' : 'bylineInfo' }).text
    prod_rate = prod_soup.find('span',{'class' : 'a-icon-alt' }).text
    prod_eval = prod_soup.find('span',{'id' : 'acrCustomerReviewText' }).text
    prod_price = prod_soup.find('span',{'id' : 'priceblock_ourprice' }).text


    result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[prod_title,prod_brand,       prod_rate,prod_eval,prod_price,prod_category]))
    return result



## Test Product Scrapping

In [219]:
driver = webdriver.Chrome(chrome_driver_path)

for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])
        print(product)

('Microsoft Surface Pro 7 PC portable (Windows 10, Écran Tactile 12.3", Intel Core i5, 8Go RAM, 128Go SSD, Platine)', 'Marque : Microsoft Surface', '4,6 sur 5 étoiles', '3 105 évaluations', '855,20 €', 'informatique')


# Scrapping Reviews Data

In [220]:
# Scrapping Reviews Data
def reviews_scrapper(driver,prod,rev_url,ds):
    driver.get(rev_url)
    rev_soup = BeautifulSoup(driver.page_source,'html.parser')
    reviews_div = rev_soup.find_all('div',{'class':reviews_div_cls})

    for review in reviews_div:
        review_title = review.find('a',{'data-hook' : 'review-title'}).text
        review_rate = review.find('i',{'data-hook' : 'review-star-rating'}).text
        review_body = review.find('span',{'data-hook' : 'review-body'}).text
        review_help = review.find('span',{'data-hook' : 'helpful-vote-statement'}).text
        rev_result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[review_title,review_rate,review_body,review_help]))
        ds.append(rev_result+prod)

    return ds

## Test Reviews Scrapping

In [221]:
ds = []
for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])
        ds = reviews_scrapper(driver,product,prod['rev_url'],ds) 


print(len(ds))
for row in ds :
    print(f'{row}\n ====================================== ')





10
('Surface Pro 7ème génération', '4,0 sur 5 étoiles', 'Je possède la Surface Pro 6 (soit la génération d\'avant), que j\'utilise aussi bien pour un usage professionnel (traitement office, logiciels spécifiques fonctionnant que sur Windows ...) que personnel (surf internet, visionnage de vidéo ...).J\'en suis très satisfait, c\'est pourquoi j\'attendais avec impatience la nouvelle génération.Pas vraiment de révolution avec cette 7ème génération, pour moi c\'est plus une mise à jour. Pour ceux ayant la génération précédente, je ne pense pas que l\'investissement soit forcément justifié (perso je change maintenant à chaque nouvelle génération de cette machine).Je retrouve dans cette machine un magnifique écran, une finition toujours aussi excellente, des performances élevées (je parle de la version i5 ici) et surtout sa "charnière" arrière toujours aussi utile.Mais aussi les défauts qui deviennent récurrents maintenant: au vu du prix, depuis le temps on 